In [ ]:
%pip install --upgrade --quiet huggingface_hub sentence_transformers

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-chroma bs4 langchainhub

In [ ]:
# Import needed libraries.
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from getpass import getpass
import os

# Keep your token secret.
HUGGINGFACEHUB_API_TOKEN = getpass("HuggingFace token: ")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

# Specify your appropriate model.
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Create our llm object.
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)

In [ ]:
# Load, chunk and index the contents of the article.
loader = WebBaseLoader("https://www.nhs.uk/mental-health/self-help/tips-and-support/raise-low-self-esteem/")
docs = loader.load()

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# In this example, we gonna use HuggingFace Embedding model.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Let's put all things togethe.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Test our RAG!
rag_chain.invoke("What is the steps to overcome the low self esteem?")

In [ ]:
# Cleanup our vectorstore after finishing.
vectorstore.delete_collection()